In [ ]:
# Explanation: Import required libraries and helper functions.
# - `openai` for API access, `dotenv` to load environment variables,
# - `base64` to encode images for inline sending, `json` for parsing,
# - `textwrap` for pretty printing responses.
import openai
from dotenv import load_dotenv  
import base64
import json
import textwrap
import os

# Explanation: Helper function to read an image file and return a base64-encoded string.
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


# Explanation: Load environment variables from a .env file if present.
load_dotenv()
# Explanation: Instantiate the OpenAI client; the key is read from the environment (.env).
openAIclient = openai.OpenAI()


# Explanation: Define the path to the example image used in the notebook.
img = "images/street_scene.jpg"




In [ ]:
# Explanation: Make a basic chat completion request sending text + an inline image.
# The model is asked: "What's in this image?" and receives the image as base64 data URL.
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Explanation: Convert the returned message content to string and print it wrapped to 120 chars.
response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))



# 1.1 Structured Output
Here, we expand upon the VLM example to request structured outputs. This approach allows for extracting 
well-organized information from images in a machine-readable format, such as JSON.

**Support Material**:
- https://platform.openai.com/docs/guides/structured-outputs


In [ ]:
# Explanation: Request a structured JSON object from the model by setting `response_format`.
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={ "type": "json_object" },# NEW!!
    temperature = 0
)

returnValue = completion.choices[0].message.content


We parse the json in a dict structure:

In [ ]:
# Explanation: Load the returned JSON string into a Python dictionary for easier access.
output = json.loads(returnValue)
# json.loads() converts JSON strings to Python objects


So we can access specific infos:

In [ ]:
# Explanation: Example of accessing a nested field (first person's description) from the parsed JSON.
output["foreground"]["people"][0]["description"]


# JSON Schema for Controlled Structured Outputs
In this section, we define a JSON schema for a more controlled and specific output from the model. 
Using this schema, we can ensure the model adheres to predefined data types and structures while describing images.In this case we will provide the json schema directly.



In [ ]:
# Explanation: Request the model to conform to a user-provided JSON schema by using `response_format` with type `json_schema`.
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={
                "type": "json_schema",    
                "json_schema": {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc.",
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc.",
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]
                    }}},
    temperature = 0
)

returnValue = completion.choices[0].message.content


In [ ]:
# Explanation: Parse the returned JSON string into a Python object.
output_image_extraction = json.loads(returnValue)


In [ ]:
# Explanation: Show the list of people extracted from the image JSON.
output_image_extraction["people"]

Alternatively: 


OpenAI SDKs for Python and JavaScript also make it easy to define object schemas using Pydantic and Zod respectively. Below, you can see how to extract information from unstructured text that conforms to a schema defined in code.


```python
from pydantic import BaseModel


class Person(BaseModel):
    position: str 
    age: int 
    activity: str 
    gender: str


class ImageExtraction(BaseModel):
    number_of_people: int 
    atmosphere: str 
    hour_of_the_day: int 
    people: list[Person] 

completion = openAIclient.beta.chat.completions.parse(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": "describe the image in detail"},
    ],
    response_format=ImageExtraction,
)

output_image_extraction = completion.choices[0].message.parsed


We can then integrate the extracted information in full or partially in a new prompt for a new extraction

In [ ]:
# Explanation: Build a prompt for an alert service that evaluates extracted scene info for danger.
# It sets a system prompt role and creates an alert prompt using the previously extracted JSON.
# `output_image_extraction` is expected to be the parsed JSON from the model.
# The prompt concatenates that JSON so the alert model can decide if medical assistance is needed.
alert_sys_prompt = " you are an experienced first aid paramedical"
alert_prompt= """Extract from the following scene analysis give to you in json format, 
if anyone might be in danger and if the Child Hospital or normal Hospital should be alerted. 
Give the a concise answer
The situation is given to you from this object: """ + str(output_image_extraction)


In [ ]:
# Explanation: Send the alert prompt to the model (two identical user messages here) and print a formatted response.
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "user", "content": alert_prompt},
        {"role": "user", "content": alert_prompt}
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))

In [ ]:
# Explanation: Ask the model to identify the youngest person and return normalized coordinates for a bounding box.
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Considering this list of people"+str(output_image_extraction["people"])+".Identify the youngest in the picture I provide and give me back their coordinates. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))


# 2. Google VLM (Gemini)
This section demonstrates the use of Google's Vision Language Model, Gemini. 
We explore basic text generation as well as its ability to analyze images and provide relevant outputs.

**Support Material**:
- https://ai.google.dev/gemini-api/docs/quickstart
- https://ai.google.dev/gemini-api/docs/text-generation
- https://ai.google.dev/gemini-api/docs/image-understanding
- https://ai.google.dev/gemini-api/docs/structured-output?example=recipe

In [ ]:
# Explanation: Set up the Google Gemini client and load the image.
%matplotlib inline
from dotenv import load_dotenv  
from google import genai
from PIL import Image
import textwrap

import json


load_dotenv()
# Instantiate the Gemini client; the key is read from the environment (.env).
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

# Path to your image
img = "images/street_scene.jpg"

Basic call:

In [ ]:
# Explanation: Example of a simple text generation call to Gemini.
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works to a 90 years old. in few words"
)

print(textwrap.fill(response.text, width=120))

and with images: 

In [ ]:
# Explanation: Open the image with PIL and send the image plus text to Gemini for description.
im = Image.open(img)

response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Describe the scene in details\n"],
                                          )

print(textwrap.fill(response.text, width=120))


Also here we can extract structured output (Gemini actually prefers pydantic syntax - let's see what happens with a schema as before)-> check limitations in https://ai.google.dev/gemini-api/docs/structured-output?example=recipe 

In [ ]:
# Explanation: Define a JSON schema for Gemini to follow and request a JSON response.
json_schema = {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc.",
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc.",
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]}}



config={
        "response_mime_type": "application/json",
        "response_json_schema": json_schema,
    }


response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Describe the scene in details, follwoing exactly the given json schema\n"],
                                          config=config
                                          )



print(response.text)

Does it match your schema?

Let's try to use Gemini to detect an object in the image and get its coordinates:


In [ ]:
# Explanation: Ask Gemini to identify the youngest person and return bounding box coordinates in a normalized 0-1000 scale.
prompt = "Identify the youngest in the picture and give me back their coordinates. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."


config={"response_mime_type": "application/json"}

response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[img, prompt],
                                          config=config
                                          )

bounding_boxes = json.loads(response.text)
print(bounding_boxes)


Gemini2+ was trained specifically for object detection/ segmentation tasks. More details: https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Spatial_understanding.ipynb


## 3.Extract Structured Infos from Hand-written note - GPT & Gemini

Let’s try **not** to extract structured information from a handwritten note (e.g., `prescription1.jpg`) using **both models**.

Consider the file: `/images/prescription1.jpg`.  
Have a look at it.

### JSON Schema
Let’s define a JSON schema for the extraction task:


In [ ]:
# Explanation: Define a JSON schema describing the prescription fields we want to extract.
json_schema_prescription = {
 "name": "prescription_extract",
"schema": {
  "type": "object",
  "properties": {
    "doctor_name": { "type": "string" },
    "patient_name": { "type": "string" },
    "patient_dob": { "type": "string" },
    "meds": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": { "type": "string" },
          "dose": { "type": "string" },
          "frequency": { "type": "string" },
          "instructions": { "type": "string" }
        },
        "required": ["name"]
      }
    },
    "signature": { "type": "boolean" }
  },
  "required": ["doctor_name", "patient_name", "meds"]
}}

Extract structured infos using Gemini: 

In [ ]:
# Explanation: Open the handwritten prescription image and ask Gemini to extract the data following the provided JSON schema.
im = Image.open("images/prescription1.jpg")

config={
        "response_mime_type": "application/json",
        "response_json_schema": json_schema_prescription,
    }


response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Extract infos from image, follwoing the given json schema.\n"],
                                          config=config
                                          )



print(response.text)

If the output is **not valid JSON** and contains extra strings, it must be **parsed** before it can be loaded into a Python dict.  
Below is an example helper function that does this.

> **Note:** Since Gemini returns a Pydantic model, you *could* use Pydantic methods to handle parsing.  
> We avoid that here to keep the workflow generally compatible across models.


In [ ]:
# Explanation: Helper to extract JSON substring from noisy model outputs and parse it into a dict.
import re
import json 
def parse_json_in_output(output):
    """
    Extracts and converts JSON-like data from the given text output to a Python dictionary.
    
    Args:
        output (str): The text output containing the JSON data.
    
    Returns:
        dict: The parsed JSON data as a Python dictionary.
    """
    # Regex to extract JSON-like portion
    json_match = re.search(r"\{.*?\}", output, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        # Fix single quotes and ensure proper JSON formatting
        json_str = json_str.replace("'", '"')  # Replace single quotes with double quotes
        try:
            # Convert the fixed JSON string into a dictionary
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError:
            return "The extracted JSON is still not valid after formatting."
    else:
        return "No JSON data found in the given output."

In [ ]:
# Explanation: Example usage (commented out) of the parsing helper.
# print(parse_json_in_output(response.text))


In [ ]:
# Explanation: Directly try to load the response text as JSON (works if output is clean JSON).
json.loads(response.text)

Now let's do the same with GPT

In [ ]:
# Explanation: Use OpenAI GPT to extract structured data from the prescription image using the previously defined schema.
im = "images/prescription1.jpg"

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(im)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={
                "type": "json_schema",   "json_schema": json_schema_prescription},
    temperature = 0
)

returnValue = completion.choices[0].message.content

In [ ]:
# Explanation: Display the raw returned value (string or JSON) from the GPT call.
returnValue

Any difference wiht the output of Gemini vs your schema? 

No need for parsing now. We load the json in a python dict structure with json.loads

In [ ]:
# Explanation: Convert the returned JSON string into a Python dictionary and print it.
print(json.loads(returnValue))